# Kannada Language Processing Workflow

This notebook demonstrates the complete workflow for processing Kannada language text using our specialized framework. Kannada (ಕನ್ನಡ) is a Dravidian language spoken primarily in Karnataka, India.

## What you'll learn:
1. **Kannada Data Collection** - Gathering text from Kannada sources
2. **Script-Specific Preprocessing** - Handling Kannada script nuances
3. **Model Training** - Optimizing models for Kannada
4. **Evaluation** - Assessing performance on Kannada tasks
5. **Cross-lingual Analysis** - Comparing with other Dravidian languages

## About Kannada
- **Script**: Kannada script (ಕನ್ನಡ ಲಿಪಿ)
- **Family**: Dravidian language family
- **Speakers**: ~44 million native speakers
- **Unicode Range**: U+0C80–U+0CFF
- **Characteristics**: Rich morphology, agglutinative, complex conjuncts

In [ ]:
# Import libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import Kannada-specific modules
from preprocessing.kannada_preprocessor import KannadaPreprocessor
from data_collection.kannada_collector import KannadaDataCollector
from models import IndianLanguageModel
from evaluation import ModelEvaluator

# Set up plotting for Kannada text
plt.rcParams['font.family'] = ['Noto Sans Kannada', 'Lohit Kannada', 'sans-serif']
plt.style.use('default')

print("✅ ಕನ್ನಡ NLP ವ್ಯವಸ್ಥೆ ಸಿದ್ಧ! (Kannada NLP System Ready!)")

## 1. Kannada Data Collection

Let's start by collecting Kannada text from various sources.

In [ ]:
# Initialize Kannada data collector
collector = KannadaDataCollector()

print("🔍 ಕನ್ನಡ ದತ್ತಾಂಶ ಸಂಗ್ರಹಣೆ ಪ್ರಾರಂಭ... (Starting Kannada data collection...)")

# For this demo, we'll use sample Kannada texts
sample_kannada_texts = [
    {
        'title': 'ಕನ್ನಡ ಭಾಷೆ ಮತ್ತು ಸಂಸ್ಕೃತಿ',
        'content': 'ಕನ್ನಡ ಭಾಷೆಯು ದ್ರಾವಿಡ ಭಾಷಾ ಕುಟುಂಬದ ಒಂದು ಪ್ರಮುಖ ಭಾಷೆಯಾಗಿದೆ. ಇದು ಕರ್ನಾಟಕ ರಾಜ್ಯದ ಅಧಿಕೃತ ಭಾಷೆಯಾಗಿದೆ ಮತ್ತು ಸುಮಾರು ೪.೪ ಕೋಟಿ ಜನರು ಮಾತನಾಡುತ್ತಾರೆ. ಕನ್ನಡ ಸಾಹಿತ್ಯವು ಸಾವಿರಾರು ವರ್ಷಗಳ ಇತಿಹಾಸವನ್ನು ಹೊಂದಿದೆ.',
        'source': 'ಕನ್ನಡ ಪ್ರಭ',
        'category': 'culture'
    },
    {
        'title': 'ಬೆಂಗಳೂರಿನ ತಂತ್ರಜ್ಞಾನ ಉದ್ಯಮ',
        'content': 'ಬೆಂಗಳೂರು ಭಾರತದ ತಂತ್ರಜ್ಞಾನ ರಾಜಧಾನಿ ಎಂದು ಹೆಸರುವಾಸಿಯಾಗಿದೆ. ಇಲ್ಲಿ ಅನೇಕ ಬಹುರಾಷ್ಟ್ರೀಯ ಕಂಪನಿಗಳು ತಮ್ಮ ಕಚೇರಿಗಳನ್ನು ಸ್ಥಾಪಿಸಿದ್ದಾರೆ. ಕೃತ್ರಿಮ ಬುದ್ಧಿಮತ್ತೆ, ಯಂತ್ರ ಕಲಿಕೆ ಮತ್ತು ಡೇಟಾ ವಿಜ್ಞಾನದಲ್ಲಿ ಇಲ್ಲಿನ ಕಂಪನಿಗಳು ಮುಂಚೂಣಿಯಲ್ಲಿವೆ.',
        'source': 'ಪ್ರಜಾವಾಣಿ',
        'category': 'technology'
    },
    {
        'title': 'ಕರ್ನಾಟಕದ ಪರಂಪರೆ',
        'content': 'ಕರ್ನಾಟಕವು ಶ್ರೀಮಂತವಾದ ಸಾಂಸ್ಕೃತಿಕ ಪರಂಪರೆಯನ್ನು ಹೊಂದಿದೆ. ಹಂಪಿ, ಮೈಸೂರು, ಬಾದಾಮಿ ಮುಂತಾದ ಸ್ಥಳಗಳು ಐತಿಹಾಸಿಕ ಮಹತ್ವವನ್ನು ಹೊಂದಿವೆ. ಯಕ್ಷಗಾನ, ಕರ್ನಾಟಕ ಸಂಗೀತ, ಮತ್ತು ಭರತನಾಟ್ಯ ಇಲ್ಲಿನ ಪ್ರಮುಖ ಕಲಾ ಪ್ರಕಾರಗಳಾಗಿವೆ.',
        'source': 'ಉದಯವಾಣಿ',
        'category': 'heritage'
    },
    {
        'title': 'ಆರೋಗ್ಯ ಮತ್ತು ಆಯುರ್ವೇದ',
        'content': 'ಆಯುರ್ವೇದವು ಭಾರತದ ಪ್ರಾಚೀನ ವೈದ್ಯ ಶಾಸ್ತ್ರವಾಗಿದೆ. ಕರ್ನಾಟಕದಲ್ಲಿ ಅನೇಕ ಆಯುರ್ವೇದ ಕಾಲೇಜುಗಳು ಮತ್ತು ಆಸ್ಪತ್ರೆಗಳಿವೆ. ಪ್ರಕೃತಿಯ ಔಷಧಿಗಳು ಮತ್ತು ಯೋಗದ ಮೂಲಕ ಆರೋಗ್ಯವನ್ನು ಕಾಪಾಡುವ ಸಂಪ್ರದಾಯವಿದೆ.',
        'source': 'ವಿಜಯ ಕರ್ನಾಟಕ',
        'category': 'health'
    },
    {
        'title': 'ಕರ್ನಾಟಕದ ಕೃಷಿ',
        'content': 'ಕರ್ನಾಟಕ ರಾಜ್ಯವು ಕೃಷಿಯಲ್ಲಿ ಪ್ರಮುಖ ಸ್ಥಾನ ಹೊಂದಿದೆ. ಇಲ್ಲಿ ಅಕ್ಕಿ, ರಾಗಿ, ಸಕ್ಕರೆ ಕಬ್ಬು, ಕಾಫಿ ಮತ್ತು ಮೆಣಸಿನಕಾಯಿ ಬೆಳೆಯುತ್ತದೆ. ಕೊಡಗು ಜಿಲ್ಲೆಯು ಕಾಫಿ ಉತ್ಪಾದನೆಗೆ ಪ್ರಸಿದ್ಧವಾಗಿದೆ. ಆಧುನಿಕ ಕೃಷಿ ತಂತ್ರಗಳ ಬಳಕೆಯಿಂದ ಉತ್ಪಾದನೆ ಹೆಚ್ಚಾಗುತ್ತಿದೆ.',
        'source': 'ಸಂಯುಕ್ತ ಕರ್ನಾಟಕ',
        'category': 'agriculture'
    }
]

# Create DataFrame
kannada_df = pd.DataFrame(sample_kannada_texts)
kannada_df['timestamp'] = pd.Timestamp.now()
kannada_df['language'] = 'kn'
kannada_df['text_length'] = kannada_df['content'].str.len()
kannada_df['word_count'] = kannada_df['content'].str.split().str.len()

print(f"📊 ಸಂಗ್ರಹಣೆ ಪೂರ್ಣ! (Collection Complete!)")
print(f"Total articles: {len(kannada_df)}")
print(f"Average text length: {kannada_df['text_length'].mean():.0f} characters")
print(f"Total words: {kannada_df['word_count'].sum()}")

# Show sample
print(f"\n📰 ಮಾದರಿ ಲೇಖನ (Sample Article):")
sample = kannada_df.iloc[0]
print(f"Title: {sample['title']}")
print(f"Content: {sample['content'][:150]}...")
print(f"Category: {sample['category']}")

## 2. Kannada Text Preprocessing

Now let's preprocess the Kannada text using our specialized preprocessor.

In [ ]:
# Initialize Kannada preprocessor
preprocessor = KannadaPreprocessor()

print("🧹 ಕನ್ನಡ ಪಾಠ್ಯ ಪ್ರಕ್ರಿಯೆ ಪ್ರಾರಂಭ... (Starting Kannada text processing...)")

# Test with a sample text
sample_text = kannada_df.iloc[1]['content']
print(f"\n📝 ಮೂಲ ಪಾಠ್ಯ (Original Text):")
print(sample_text)

# Clean the text
cleaned_text = preprocessor.clean_kannada_text(
    sample_text,
    normalize_unicode=True,
    normalize_digits=True,
    handle_virama=True,
    preserve_conjuncts=True
)

print(f"\n✨ ಶುದ್ಧೀಕರಿಸಿದ ಪಾಠ್ಯ (Cleaned Text):")
print(cleaned_text)

# Get detailed statistics
stats = preprocessor.get_kannada_text_statistics(sample_text)
print(f"\n📊 ಪಾಠ್ಯ ಅಂಕಿಅಂಶಗಳು (Text Statistics):")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

# Tokenize words
tokens = preprocessor.tokenize_kannada_words(sample_text)
print(f"\n🔤 ಪದ ವಿಭಜನೆ (Word Tokenization):")
print(f"Total tokens: {len(tokens)}")
print(f"First 10 tokens: {tokens[:10]}")

# Morphological analysis of sample words
print(f"\n🔬 ರೂಪವಿಜ್ಞಾನ ವಿಶ್ಲೇಷಣೆ (Morphological Analysis):")
sample_words = ['ಕಂಪನಿಗಳು', 'ತಂತ್ರಜ್ಞಾನ', 'ಸ್ಥಾಪಿಸಿದ್ದಾರೆ']
for word in sample_words:
    analysis = preprocessor.analyze_kannada_morphology(word)
    print(f"  {word}: {analysis}")

## 3. Dataset Preprocessing

Let's preprocess our entire Kannada dataset.

In [ ]:
# Preprocess the entire dataset
print("⚙️ ಸಂಪೂರ್ಣ ದತ್ತಸಮೂಹ ಪ್ರಕ್ರಿಯೆ... (Processing entire dataset...)")

# Apply preprocessing to all content
processed_df = preprocessor.preprocess_kannada_dataset(
    kannada_df.copy(),
    text_column='content',
    clean_options={
        'normalize_unicode': True,
        'normalize_digits': True,
        'remove_mixed_script': False,
        'preserve_conjuncts': True,
        'handle_virama': True
    }
)

# Add additional analysis
processed_df['cleaned_word_count'] = processed_df['content'].str.split().str.len()
processed_df['cleaned_length'] = processed_df['content'].str.len()
processed_df['sentences'] = processed_df['content'].apply(
    lambda x: len(preprocessor.extract_kannada_sentences(x))
)

# Validation
processed_df['is_valid_kannada'] = processed_df['content'].apply(
    lambda x: preprocessor.validate_kannada_text(x, min_kannada_ratio=0.6)
)

print(f"✅ ಪ್ರಕ್ರಿಯೆ ಪೂರ್ಣ! (Processing Complete!)")
print(f"Valid Kannada texts: {processed_df['is_valid_kannada'].sum()}/{len(processed_df)}")
print(f"Average sentences per text: {processed_df['sentences'].mean():.1f}")
print(f"Total cleaned words: {processed_df['cleaned_word_count'].sum()}")

# Show processing comparison
comparison_df = pd.DataFrame({
    'Metric': ['Original Length', 'Cleaned Length', 'Original Words', 'Cleaned Words'],
    'Average': [
        processed_df['text_length'].mean(),
        processed_df['cleaned_length'].mean(), 
        processed_df['word_count'].mean(),
        processed_df['cleaned_word_count'].mean()
    ]
})

print(f"\n📈 ಪ್ರಕ್ರಿಯೆ ಹೋಲಿಕೆ (Processing Comparison):")
print(comparison_df.round(1))

## 4. Kannada Language Model Training

Let's create and configure a language model optimized for Kannada.

In [ ]:
# Initialize Kannada-optimized model
print("🤖 ಕನ್ನಡ ಭಾಷಾ ಮಾದರಿ ಆರಂಭ... (Initializing Kannada language model...)")

kannada_model = IndianLanguageModel(
    language='kn',
    model_type='bert',
    vocab_size=32000,  # Optimized for Kannada
    hidden_size=768,
    num_hidden_layers=8,  # Smaller for demo
    num_attention_heads=12,
    max_position_embeddings=1024,  # Longer sequences for Kannada
    dropout_prob=0.1
)

print("✅ ಮಾದರಿ ಸಿದ್ಧ! (Model Ready!)")

# Get model information
param_count = kannada_model.get_parameter_count()
print(f"\n📊 ಮಾದರಿ ಮಾಹಿತಿ (Model Information):")
for component, count in param_count.items():
    print(f"  {component}: {count:,} parameters")

print(f"\n🎯 Total parameters: {param_count['total']:,}")

# Test embeddings with Kannada sentences
test_sentences = [
    "ಕನ್ನಡ ಭಾಷೆ ಬಹಳ ಸುಂದರವಾಗಿದೆ.",
    "ಬೆಂಗಳೂರು ತಂತ್ರಜ್ಞಾನ ನಗರವಾಗಿದೆ.", 
    "ನಾವು ಕೃತ್ರಿಮ ಬುದ್ಧಿಮತ್ತೆಯನ್ನು ಅಧ್ಯಯನ ಮಾಡುತ್ತಿದ್ದೇವೆ.",
    "ಕರ್ನಾಟಕದ ಸಂಸ್ಕೃತಿ ಶ್ರೀಮಂತವಾಗಿದೆ."
]

print(f"\n🧠 ಭಾಷಾ ಮಾದರಿ ಪರೀಕ್ಷೆ (Language Model Testing):")
embeddings = kannada_model.get_embeddings(test_sentences, language='kn')
print(f"Embeddings shape: {embeddings.shape}")
print(f"Sample embedding (first 5 dims): {embeddings[0][:5].numpy()}")

# Calculate similarities between sentences
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(embeddings)

print(f"\n🔄 ವಾಕ್ಯ ಸಾಮ್ಯತೆಗಳು (Sentence Similarities):")
for i, sent1 in enumerate(test_sentences):
    for j, sent2 in enumerate(test_sentences[i+1:], i+1):
        sim = similarities[i][j]
        print(f"  {sim:.3f}: '{sent1}' ↔ '{sent2}'")

## 5. Model Evaluation on Kannada Tasks

Let's evaluate our model's performance on various Kannada NLP tasks.

In [ ]:
# Initialize evaluator for Kannada
evaluator = ModelEvaluator(
    model=kannada_model,
    language='kn',
    device='cpu'
)

print("📊 ಕನ್ನಡ ಮಾದರಿ ಮೌಲ್ಯಮಾಪನ... (Kannada Model Evaluation...)")

# Create evaluation dataset from our processed texts
eval_texts = processed_df['content'].tolist()
eval_categories = processed_df['category'].tolist()

# Create label mapping for categories
category_to_label = {cat: idx for idx, cat in enumerate(set(eval_categories))}
label_to_category = {v: k for k, v in category_to_label.items()}
eval_labels = [category_to_label[cat] for cat in eval_categories]

print(f"📋 ಮೌಲ್ಯಮಾಪನ ದತ್ತಸಮೂಹ (Evaluation Dataset):")
print(f"  Texts: {len(eval_texts)}")
print(f"  Categories: {list(category_to_label.keys())}")
print(f"  Labels: {eval_labels}")

# Text classification evaluation
print(f"\n🎯 ಪಾಠ್ಯ ವರ್ಗೀಕರಣ ಮೌಲ್ಯಮಾಪನ (Text Classification Evaluation):")
classification_results = evaluator.evaluate_text_classification(
    texts=eval_texts,
    labels=eval_labels,
    task_name="kannada_category_classification"
)

print(f"Classification Results:")
for metric, value in classification_results.items():
    if isinstance(value, (int, float)):
        print(f"  {metric}: {value:.4f}")
    elif metric not in ['classification_report']:
        print(f"  {metric}: {value}")

# Semantic similarity evaluation
print(f"\n🔄 ಅರ್ಥಗತ ಸಾಮ್ಯತೆ ಮೌಲ್ಯಮಾಪನ (Semantic Similarity Evaluation):")

# Create pairs of similar and dissimilar Kannada sentences
similarity_pairs = [
    ("ಕನ್ನಡ ಭಾಷೆ ಸುಂದರವಾಗಿದೆ", "ಕನ್ನಡ ಭಾಷೆ ಚೆಂದವಾಗಿದೆ"),  # Similar
    ("ಬೆಂಗಳೂರು ದೊಡ್ಡ ನಗರ", "ಬೆಂಗಳೂರು ಮಹಾನಗರ"),  # Similar
    ("ತಂತ್ರಜ್ಞಾನ ಅಭಿವೃದ್ಧಿ", "ಕೃಷಿ ಉತ್ಪಾದನೆ"),  # Different
    ("ಸಂಸ್ಕೃತಿ ಮತ್ತು ಕಲೆ", "ಆರೋಗ್ಯ ಮತ್ತು ಔಷಧಿ")  # Different
]

similarity_scores = [0.8, 0.7, 0.2, 0.1]  # Expected similarity scores

similarity_results = evaluator.evaluate_semantic_similarity(
    text_pairs=similarity_pairs,
    similarity_scores=similarity_scores
)

print(f"Semantic Similarity Results:")
for metric, value in similarity_results.items():
    print(f"  {metric}: {value:.4f}")

# Show individual pair similarities
print(f"\n📝 ವ್ಯಕ್ತಿಗತ ಜೋಡಿ ಸಾಮ್ಯತೆಗಳು (Individual Pair Similarities):")
for i, (text1, text2) in enumerate(similarity_pairs):
    emb1 = kannada_model.get_embeddings([text1], language='kn')
    emb2 = kannada_model.get_embeddings([text2], language='kn')
    sim = cosine_similarity(emb1, emb2)[0, 0]
    print(f"  {sim:.3f}: '{text1}' ↔ '{text2}'")

## 6. Cross-lingual Analysis with Dravidian Languages

Let's evaluate cross-lingual capabilities with other Dravidian languages.

In [ ]:
print("🌍 ದ್ರಾವಿಡ ಭಾಷೆಗಳ ಅಂತರ್-ಭಾಷೆ ವಿಶ್ಲೇಷಣೆ (Cross-lingual Analysis with Dravidian Languages)")

# Multilingual data with Dravidian languages
dravidian_data = {
    'kn': {  # Kannada
        'texts': [
            "ಇದು ಒಂದು ಚೆಂದದ ದಿನ.",
            "ನಾನು ಕನ್ನಡ ಭಾಷೆಯನ್ನು ಪ್ರೀತಿಸುತ್ತೇನೆ.",
            "ತಂತ್ರಜ್ಞಾನ ನಮ್ಮ ಭವಿಷ್ಯ."
        ],
        'labels': [1, 1, 1]  # Positive sentiment
    },
    'ta': {  # Tamil  
        'texts': [
            "இது ஒரு அழகான நாள்.",  # This is a beautiful day
            "நான் தமிழ் மொழியை நேசிக்கிறேன்.",  # I love Tamil language
            "தொழில்நுட்பம் நமது எதிர்காலம்."  # Technology is our future
        ],
        'labels': [1, 1, 1]  # Positive sentiment
    },
    'te': {  # Telugu
        'texts': [
            "ఇది ఒక అందమైన రోజు.",  # This is a beautiful day
            "నేను తెలుగు భాషను ప్రేమిస్తున్నాను.",  # I love Telugu language  
            "సాంకేతికత మన భవిష్యత్తు."  # Technology is our future
        ],
        'labels': [1, 1, 1]  # Positive sentiment
    },
    'ml': {  # Malayalam
        'texts': [
            "ഇത് മനോഹരമായ ദിനമാണ്.",  # This is a beautiful day
            "ഞാൻ മലയാളം ഭാഷയെ സ്നേഹിക്കുന്നു.",  # I love Malayalam language
            "സാങ്കേതികവിദ്യ നമ്മുടെ ഭാവിയാണ്."  # Technology is our future
        ],
        'labels': [1, 1, 1]  # Positive sentiment
    }
}

# Multilingual evaluation
multilingual_results = evaluator.evaluate_multilingual_capabilities(dravidian_data)

print(f"\n🗺️ ಬಹುಭಾಷೆ ಫಲಿತಾಂಶಗಳು (Multilingual Results):")
print(f"  Supported languages: {multilingual_results['supported_languages']}")
print(f"  Consistency score: {multilingual_results['consistency_score']:.4f}")
print(f"  Number of languages: {multilingual_results['num_languages']}")

# Language-specific results
print(f"\n📊 ಭಾಷಾ-ನಿರ್ದಿಷ್ಟ ಫಲಿತಾಂಶಗಳು (Language-specific Results):")
for lang, results in multilingual_results['language_results'].items():
    lang_names = {'kn': 'ಕನ್ನಡ', 'ta': 'ತಮಿಳು', 'te': 'ತೆಲುಗು', 'ml': 'ಮಲಯಾಳಂ'}
    print(f"\n  {lang_names.get(lang, lang).upper()} ({lang}):")
    for metric, value in results.items():
        if isinstance(value, (int, float)):
            print(f"    {metric}: {value:.4f}")
        else:
            print(f"    {metric}: {value}")

# Cross-lingual transfer: Kannada → Tamil
print(f"\n🔄 ಅಂತರ್-ಭಾಷೆ ವರ್ಗಾವಣೆ: ಕನ್ನಡ → ತಮಿಳು (Cross-lingual Transfer: Kannada → Tamil)")

cross_lingual_results = evaluator.evaluate_cross_lingual_transfer(
    source_data=dravidian_data['kn'],
    target_data=dravidian_data['ta'],
    source_lang='kn',
    target_lang='ta'
)

print(f"Cross-lingual Transfer Results:")
for metric, value in cross_lingual_results.items():
    if isinstance(value, (int, float)):
        print(f"  {metric}: {value:.4f}")
    else:
        print(f"  {metric}: {value}")

## 7. Kannada Text Analysis and Visualization

Let's create some visualizations specific to Kannada text analysis.

In [ ]:
print("📊 ಕನ್ನಡ ಪಾಠ್ಯ ವಿಶ್ಲೇಷಣೆ ಮತ್ತು ದೃಶ್ಯೀಕರಣ (Kannada Text Analysis and Visualization)")

# Analyze character distribution in Kannada texts
all_kannada_text = ' '.join(processed_df['content'])
char_analysis = {}

# Count different types of characters
kannada_chars = sum(1 for c in all_kannada_text if 0x0C80 <= ord(c) <= 0x0CFF)
latin_chars = sum(1 for c in all_kannada_text if c.isalpha() and ord(c) < 128)
digits = sum(1 for c in all_kannada_text if c.isdigit())
spaces = sum(1 for c in all_kannada_text if c.isspace())
punctuation = sum(1 for c in all_kannada_text if c in '.,!?;:')

char_distribution = {
    'ಕನ್ನಡ ಅಕ್ಷರಗಳು': kannada_chars,
    'ಲ್ಯಾಟಿನ್ ಅಕ್ಷರಗಳು': latin_chars,
    'ಅಂಕೆಗಳು': digits,
    'ಖಾಲಿ ಜಾಗಗಳು': spaces,
    'ವಿರಾಮ ಚಿಹ್ನೆಗಳು': punctuation
}

print(f"\n🔤 ಅಕ್ಷರ ವಿತರಣೆ (Character Distribution):")
for char_type, count in char_distribution.items():
    percentage = (count / len(all_kannada_text)) * 100
    print(f"  {char_type}: {count:,} ({percentage:.1f}%)")

# Word length analysis
all_words = []
for text in processed_df['content']:
    words = preprocessor.tokenize_kannada_words(text)
    all_words.extend(words)

word_lengths = [len(word) for word in all_words]

print(f"\n📏 ಪದ ಉದ್ದದ ವಿಶ್ಲೇಷಣೆ (Word Length Analysis):")
print(f"  Total words: {len(all_words):,}")
print(f"  Average word length: {np.mean(word_lengths):.1f} characters")
print(f"  Shortest word: {min(all_words, key=len)} ({len(min(all_words, key=len))} chars)")
print(f"  Longest word: {max(all_words, key=len)} ({len(max(all_words, key=len))} chars)")

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ಕನ್ನಡ ಪಾಠ್ಯ ವಿಶ್ಲೇಷಣೆ (Kannada Text Analysis)', fontsize=16)

# Character distribution pie chart
axes[0, 0].pie(char_distribution.values(), labels=char_distribution.keys(), autopct='%1.1f%%')
axes[0, 0].set_title('ಅಕ್ಷರ ವಿತರಣೆ (Character Distribution)')

# Word length histogram
axes[0, 1].hist(word_lengths, bins=20, alpha=0.7, color='skyblue')
axes[0, 1].set_title('ಪದ ಉದ್ದದ ವಿತರಣೆ (Word Length Distribution)')
axes[0, 1].set_xlabel('Word Length (characters)')
axes[0, 1].set_ylabel('Frequency')

# Category distribution
category_counts = processed_df['category'].value_counts()
axes[1, 0].bar(category_counts.index, category_counts.values)
axes[1, 0].set_title('ವಿಷಯ ವಿತರಣೆ (Category Distribution)')
axes[1, 0].set_xlabel('Category')
axes[1, 0].set_ylabel('Count')
plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45)

# Text length distribution
axes[1, 1].hist(processed_df['cleaned_length'], bins=10, alpha=0.7, color='lightgreen')
axes[1, 1].set_title('ಪಾಠ್ಯ ಉದ್ದದ ವಿತರಣೆ (Text Length Distribution)')
axes[1, 1].set_xlabel('Text Length (characters)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Most common words
from collections import Counter
word_freq = Counter(all_words)
common_words = word_freq.most_common(10)

print(f"\n🔝 ಸಾಮಾನ್ಯ ಪದಗಳು (Most Common Words):")
for word, freq in common_words:
    print(f"  {word}: {freq} times")

## 8. Generate Comprehensive Kannada Report

Let's generate a comprehensive evaluation report for our Kannada model.

In [ ]:
# Generate comprehensive evaluation report
print("📋 ಸಮಗ್ರ ಕನ್ನಡ ಮೌಲ್ಯಮಾಪನ ವರದಿ ತಯಾರಿಕೆ... (Generating Comprehensive Kannada Evaluation Report...)")

report = evaluator.generate_evaluation_report(
    output_path='../data/kannada_evaluation_report'
)

print("\n" + "="*80)
print("ಕನ್ನಡ ಭಾಷಾ ಮಾದರಿ ಮೌಲ್ಯಮಾಪನ ವರದಿ (KANNADA LANGUAGE MODEL EVALUATION REPORT)")
print("="*80)
print(report[:1500] + "..." if len(report) > 1500 else report)

print("\n✅ ವರದಿ ಉಳಿಸಲಾಗಿದೆ: ../data/kannada_evaluation_report.json ಮತ್ತು .txt ಫೈಲ್‌ಗಳಲ್ಲಿ")
print("✅ Report saved to: ../data/kannada_evaluation_report.json and .txt files")

## 9. Save Kannada Model

Finally, let's save our trained Kannada model for future use.

In [ ]:
# Save the Kannada-optimized model
kannada_model_path = '../data/models/kannada/kannada_language_model'

print(f"💾 ಕನ್ನಡ ಮಾದರಿಯನ್ನು ಉಳಿಸಲಾಗುತ್ತಿದೆ... (Saving Kannada model to: {kannada_model_path})")

try:
    kannada_model.save_model(kannada_model_path)
    print("✅ ಮಾದರಿ ಯಶಸ್ವಿಯಾಗಿ ಉಳಿಸಲಾಗಿದೆ! (Model saved successfully!)")
    
    # Test loading the model
    print("🔄 ಮಾದರಿ ಲೋಡ್ ಮಾಡುವಿಕೆಯನ್ನು ಪರೀಕ್ಷಿಸಲಾಗುತ್ತಿದೆ... (Testing model loading...)")
    loaded_model = IndianLanguageModel.load_model(kannada_model_path)
    print("✅ ಮಾದರಿ ಯಶಸ್ವಿಯಾಗಿ ಲೋಡ್ ಆಗಿದೆ! (Model loaded successfully!)")
    
    # Verify the loaded model works with Kannada
    test_kannada_text = "ಇದು ಒಂದು ಪರೀಕ್ಷಾ ವಾಕ್ಯವಾಗಿದೆ."  # This is a test sentence
    test_embeddings = loaded_model.get_embeddings([test_kannada_text], language='kn')
    print(f"✅ ಲೋಡ್ ಮಾಡಿದ ಮಾದರಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತಿದೆ! ಎಂಬೆಡಿಂಗ್ ಆಕಾರ: {test_embeddings.shape}")
    print(f"✅ Loaded model working! Embedding shape: {test_embeddings.shape}")
    
except Exception as e:
    print(f"❌ ಮಾದರಿ ಉಳಿಸುವಲ್ಲಿ/ಲೋಡ್ ಮಾಡುವಲ್ಲಿ ದೋಷ: {e}")
    print(f"❌ Error saving/loading model: {e}")

# Save the processed dataset too
dataset_path = '../data/processed/kannada_dataset.csv'
processed_df.to_csv(dataset_path, index=False, encoding='utf-8')
print(f"\n💾 ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಿದ ದತ್ತಸಮೂಹ ಉಳಿಸಲಾಗಿದೆ: {dataset_path}")
print(f"💾 Processed dataset saved to: {dataset_path}")

## ಸಾರಾಂಶ (Summary)

🎉 **ಅಭಿನಂದನೆಗಳು! (Congratulations!)** 

You've successfully completed the Kannada language processing workflow:

### ಪೂರ್ಣಗೊಂಡ ಕಾರ್ಯಗಳು (Completed Tasks):

1. ✅ **ದತ್ತ ಸಂಗ್ರಹಣೆ (Data Collection)**: Collected and curated Kannada text from various domains
2. ✅ **ಪ್ರೀ-ಪ್ರೊಸೆಸಿಂಗ್ (Preprocessing)**: Applied Kannada-specific text cleaning and normalization
3. ✅ **ಮಾದರಿ ತರಬೇತಿ (Model Training)**: Configured and initialized model optimized for Kannada
4. ✅ **ಮೌಲ್ಯಮಾಪನ (Evaluation)**: Assessed model performance on Kannada NLP tasks
5. ✅ **ಅಂತರ್-ಭಾಷೆ ವಿಶ್ಲೇಷಣೆ (Cross-lingual Analysis)**: Compared with other Dravidian languages
6. ✅ **ದೃಶ್ಯೀಕರಣ (Visualization)**: Created Kannada text analysis visualizations
7. ✅ **ವರದಿ ತಯಾರಿಕೆ (Report Generation)**: Generated comprehensive evaluation reports
8. ✅ **ಮಾದರಿ ಉಳಿಕೆ (Model Saving)**: Saved the model for future use

### ಮುಂದಿನ ಹಂತಗಳು (Next Steps):

- **ದೊಡ್ಡ ದತ್ತಸಮೂಹದಲ್ಲಿ ತರಬೇತಿ**: Train on larger Kannada datasets
- **ವಿಶೇಷ ಕಾರ್ಯಗಳು**: Implement specific tasks like POS tagging, NER for Kannada
- **ಸಾಹಿತ್ಯ ವಿಶ್ಲೇಷಣೆ**: Analyze classical Kannada literature
- **ಕನ್ನಡ-ಇಂಗ್ಲಿಷ್ ಅನುವಾದ**: Develop translation models
- **ಧ್ವನಿ ಸಂಸ್ಕರಣೆ**: Integrate with Kannada speech processing

### ಸಂಪನ್ಮೂಲಗಳು (Resources):

- [ಕನ್ನಡ ಸಾಹಿತ್ಯ ಪರಿಷತ್ (Kannada Sahitya Parishat)](https://kannadasahityaparishat.org/)
- [ಕನ್ನಡ ವಿಕಿಪೀಡಿಯಾ (Kannada Wikipedia)](https://kn.wikipedia.org/)
- [AI4Bharat Kannada Resources](https://ai4bharat.org/)
- [IndicNLP Library](https://github.com/anoopkunchukuttan/indic_nlp_library)

---

**ಕನ್ನಡ ಭಾಷೆಯ ಡಿಜಿಟಲ್ ಭವಿಷ್ಯದ ನಿರ್ಮಾಣದಲ್ಲಿ ಪಾಲ್ಗೊಳ್ಳಿ!** 🌟

**Join in building the digital future of the Kannada language!** 🇮🇳✨